# Первая часть кода. Парсинг сайта

Здесь мы сделаем общий код, куда будем вставлять страницы сайта paroles.net и выкачивать с них ссылки на песни. Затем ссылки на песни будут обработаны и мы выкачаем непосредственно сам текст.

In [7]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import json
from datetime import datetime

In [95]:
BASE_URL = "https://www.paroles.net/dranem"
OUTPUT_JSON = "Dranem_songs.json"  #Имя выходного файла
DELAY = 1  #Задержка между запросами
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'fr-FR,fr;q=0.9'
}

In [29]:
#Получает все ссылки на песни артиста
def get_song_links(artist_url):
    try:
        response = requests.get(artist_url, headers=HEADERS, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        left_div = soup.select_one('#main > div.box > div > div > div.center-on-mobile.box-content.left')
        right_div = soup.select_one('#main > div.box > div > div > div.center-on-mobile.box-content.right')

        links = []

        def extract_links(div):
            if div:
                for a in div.find_all('a', href=True):
                    href = a['href'].strip()
                    if href and not href.startswith(('javascript:', '#')):
                        links.append(urljoin(artist_url, href))

        extract_links(left_div)
        extract_links(right_div)

        return sorted(list(set(links))), None

    except Exception as e:
        return [], str(e)

In [30]:
#Извлекает текст песни со страницы
def get_song_text(song_url):
    try:
        time.sleep(DELAY)
        response = requests.get(song_url, headers=HEADERS, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        song_div = soup.find('div', class_='song-text')
        if not song_div:
            return None, "Не найден текст песни"

        #Очищаем текст от лишних элементов
        for br in song_div.find_all('br'):
            br.replace_with('\n')
        for span in song_div.find_all('span', class_='comment'):
            span.decompose()

        song_text = song_div.get_text(separator='\n', strip=True)
        title = soup.find('h1').get_text(strip=True) if soup.find('h1') else "Без названия"

        #Извлекаем возможные дополнительные данные
        meta = {
            'title': title,
            'url': song_url,
            'text': song_text,
            'scraped_date': datetime.now().isoformat()
        }

        #Попробуем найти год песни (если есть)
        year_tag = soup.find('span', class_='year')
        if year_tag:
            meta['year'] = year_tag.get_text(strip=True)

        return meta, None

    except Exception as e:
        return None, str(e)

In [31]:
#Сохраняет данные в JSON-файл с красивым форматированием
def save_to_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2, sort_keys=True)
    print(f"Данные сохранены в {filename}")


In [96]:
print("Начинаем сбор песен...")

#Получаем все ссылки на песни
song_links, error = get_song_links(BASE_URL)
if error:
    print(f"Ошибка при получении списка песен: {error}")
    exit()

print(f"Найдено {len(song_links)} песен")

#Собираем тексты песен
songs_data = {
    'metadata': {
        'source': BASE_URL,
        'scraped_at': datetime.now().isoformat(),
        'total_songs': len(song_links)
    },
    'songs': []
}

errors = []

for i, song_url in enumerate(song_links, 1):
    print(f"Обрабатываем песню {i}/{len(song_links)}: {song_url}")

    song_meta, error = get_song_text(song_url)

    if error:
        errors.append({'url': song_url, 'error': error})
        continue

    if song_meta:
        songs_data['songs'].append(song_meta)

#Добавляем информацию об ошибках
if errors:
    songs_data['metadata']['error_count'] = len(errors)
    songs_data['errors'] = errors[:10]  #Сохраняем первые 10 ошибок

#Сохраняем результаты
save_to_json(songs_data, OUTPUT_JSON)

#Выводим статистику
print("\nРезультаты:")
print(f"Успешно собрано: {len(songs_data['songs'])} песен")
print(f"Ошибок: {len(errors)}")
print(f"Данные сохранены в {OUTPUT_JSON}")

Начинаем сбор песен...
Найдено 23 песен
Обрабатываем песню 1/23: https://www.paroles.net/dranem/paroles-ah-les-chinois
Обрабатываем песню 2/23: https://www.paroles.net/dranem/paroles-avec-son-pot-d-geranium
Обрабатываем песню 3/23: https://www.paroles.net/dranem/paroles-cache-ton-piano
Обрабатываем песню 4/23: https://www.paroles.net/dranem/paroles-chanson-des-petits-bateaux
Обрабатываем песню 5/23: https://www.paroles.net/dranem/paroles-encore-cinquante-centimes
Обрабатываем песню 6/23: https://www.paroles.net/dranem/paroles-folle-complainte
Обрабатываем песню 7/23: https://www.paroles.net/dranem/paroles-jambe-en-bois-la
Обрабатываем песню 8/23: https://www.paroles.net/dranem/paroles-l-argent
Обрабатываем песню 9/23: https://www.paroles.net/dranem/paroles-l-article-214-elle-est-toujours-derriere
Обрабатываем песню 10/23: https://www.paroles.net/dranem/paroles-la-chanson-du-doge
Обрабатываем песню 11/23: https://www.paroles.net/dranem/paroles-la-mere-du-pere
Обрабатываем песню 12/23: h

# Вторая часть кода, где мы добавим все данные в БД

In [60]:
import sqlite3
import os
import re

In [50]:
def get_db_connection():
    conn = sqlite3.connect('/content/Chanson_françaises_XX (4).db')
    conn.row_factory = sqlite3.Row
    return conn

In [58]:
#Извлекает имя артиста из названия файла
def extract_artist_name(filename):
    base_name = os.path.splitext(filename)[0]
    name = base_name.replace('_', ' ')
    name = re.sub(r'\s+songs?$', '', name, flags=re.IGNORECASE)
    return name.strip()

In [51]:
#Очищает текст песни от лишних слов
def clean_song_text(text):
    if text.startswith("Paroles de la chanson"):
        lines = text.split('\n', 1)
        return lines[1].strip() if len(lines) > 1 else text
    return text

In [52]:
#Очищает название песни от лишнего
def clean_song_title(title):
    return title.replace("Paroles de la chanson", "").strip()

In [53]:
#Возвращает ID автора, создает запись если не существует
def get_or_create_author(conn, author_name):
    cursor = conn.cursor()
    cursor.execute("SELECT id FROM author WHERE name = ?", (author_name,))
    if author := cursor.fetchone():
        return author['id']

    cursor.execute("INSERT INTO author (name) VALUES (?)", (author_name,))
    conn.commit()
    return cursor.lastrowid

In [64]:
#Вставляет песню в базу данных
def insert_song(conn, author_id, title, text):
    cursor = conn.cursor()
    cursor.execute(
        """INSERT INTO song (author_id, title, lyrics)
           VALUES (?, ?, ?)""",
        (author_id, clean_song_title(title), clean_song_text(text))
    )
    conn.commit()
    return cursor.lastrowid

In [68]:
#Основная функция для обработки JSON файла
def process_songs_from_json(json_file_path):
    # Извлекаем имя артиста из названия файла
    artist_name = extract_artist_name(os.path.basename(json_file_path))

    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    conn = get_db_connection()
    try:
        author_id = get_or_create_author(conn, artist_name)
        print(f"Обработка песен для автора: {artist_name} (ID: {author_id})")

        success_count = 0
        for song in data['songs']:
            try:
                song_id = insert_song(conn, author_id, song['title'], song['text'])
                print(f"Добавлена: {clean_song_title(song['title'])}")
                success_count += 1
            except Exception as e:
                print(f"Ошибка при добавлении песни: {str(e)}")
                continue

        print(f"Успешно добавлено песен: {success_count}/{len(data['songs'])}")

    except Exception as e:
        print(f"Ошибка обработки файла: {str(e)}")
    finally:
        conn.close()

In [97]:
json_file_path = "/content/Dranem_songs.json"
process_songs_from_json(json_file_path)

Обработка песен для автора: Dranem (ID: 85)
Добавлена: Ah, les chinois !parDranem
Добавлена: Avec Son Pot D' GeraniumparDranem
Добавлена: Cache Ton PianoparDranem
Добавлена: Chanson des petits bateauxparDranem
Добавлена: Encore cinquante centimesparDranem
Добавлена: Folle complainteparDranem
Добавлена: Jambe En Bois (la)parDranem
Добавлена: L'argentparDranem
Добавлена: L'article 214 (elle est toujours derrière)parDranem
Добавлена: La chanson du dogeparDranem
Добавлена: La mère du pèreparDranem
Добавлена: La vigne aux moineaux (ils sont dans les vignes, les moineaux)parDranem
Добавлена: Le moment de m'en allerparDranem
Добавлена: Le Trou De Mon QuaiparDranem
Добавлена: Le vase de soissonsparDranem
Добавлена: Les p'tits pois (chant patriotique)parDranem
Добавлена: Nous nous plûmesparDranem
Добавлена: Quand on n'en a pasparDranem
Добавлена: RaymondeparDranem
Добавлена: Romance subjonctiveparDranem
Добавлена: Un millionparDranem
Добавлена: V'là l'rétameur !parDranem
Добавлена: Y n' faut pa

Случился небольшой промах и в название песни попал автор. Сейчас исправим.

In [98]:
def clean_title(title, artist_name):
    #Создаем паттерн для поиска: 'par' + имя автора (с пробелами)
    pattern = r'\s*par' + re.escape(artist_name) + r'\s*$'
    #Удаляем этот паттерн из названия песни
    return re.sub(pattern, '', title, flags=re.IGNORECASE).strip()

In [99]:
def fix_existing_songs():
    conn = get_db_connection()
    cursor = conn.cursor()

    try:
        #Получаем все песни с именами авторов
        cursor.execute("""
            SELECT s.id, s.title, a.name as artist_name
            FROM song s
            JOIN author a ON s.author_id = a.id
            WHERE s.title LIKE '%par%'
        """)

        updated_count = 0
        potential_matches = cursor.rowcount

        print("Анализ песен в базе данных:")
        print("-" * 50)

        for song in cursor.fetchall():
            original_title = song['title']
            artist_name = song['artist_name']
            search_pattern = 'par' + artist_name


            cleaned_title = clean_title(original_title, artist_name)

            if cleaned_title != original_title:
                cursor.execute("""
                    UPDATE song
                    SET title = ?
                    WHERE id = ?
                """, (cleaned_title, song['id']))
                updated_count += 1

        conn.commit()
        print("\n" + "=" * 50)
        print(f"Всего найдено песен с 'par': {potential_matches}")
        print(f"Фактически обновлено записей: {updated_count}")

    except Exception as e:
        print(f"\nОшибка при обновлении базы данных: {str(e)}")
        conn.rollback()
    finally:
        conn.close()

In [100]:
fix_existing_songs()

Анализ песен в базе данных:
--------------------------------------------------

Всего найдено песен с 'par': -1
Фактически обновлено записей: 23
